https://medium.com/enterprise-rag/an-introduction-to-rag-and-simple-complex-rag-9c3aa9bd017b
https://medium.com/@himanshuit3036/day-3-building-our-first-rag-e5b49b6cb6f6


In [16]:
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [17]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# split example to  not use splitter
splits = [
    "The fox is a carnivore that is part of the Canidae family. It is a small to medium-sized animal that is known for its cunning and intelligence. Foxes are found in a variety of habitats, including forests, grasslands, and urban areas. They are opportunistic feeders and will eat a wide range of foods, including small mammals, birds, insects, and fruit. Foxes are known for their distinctive red fur and bushy tail, which they use for communication and balance. They are solitary animals that are most active at night, although they can also be seen during the day. Foxes are found on every continent except Antarctica and are one of the most widespread carnivores in the world.",
    "The red fox is the most common species of fox and is found throughout the Northern Hemisphere. It is known for its adaptability and ability to thrive in a wide range of habitats. The red fox is a highly intelligent animal that is capable of solving complex problems and adapting to new environments. It is a skilled hunter and can catch a wide range of prey, including rabbits, rodents, and birds. The red fox is a social animal that lives in family groups known as packs. These packs are led by a dominant",
    "The gray fox is a species of fox that is found in North and South America. It is known for its distinctive gray fur and black-tipped tail. The gray fox is a solitary animal that is most active at night. It is an opportunistic feeder and will eat a wide range of foods, including small mammals, birds, insects, and fruit. The gray fox is a skilled climber and can climb trees to escape predators or reach food sources. It is also an excellent swimmer and can swim long distances to reach new habitats. The gray fox is found in a variety of habitats, including forests, grasslands, and urban areas. It is one of the most widespread species of fox and is found throughout North and South America.",
    "The arctic fox is a species of fox that is found in the Arctic regions of North America, Europe, and Asia. It is known for its thick white fur and bushy tail, which help it to survive in the cold Arctic climate. The arctic fox is a highly adaptable animal that is capable of surviving in a wide range of habitats, including tundra, mountains, and coastal areas. It is a skilled hunter and can catch a wide range of prey, including lemmings, voles, and birds. The arctic fox is a social animal that lives in family groups known as packs. These packs are led by a dominant"
]



In [18]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [19]:
vectorstore = FAISS.from_texts(texts=splits, embedding=OpenAIEmbeddings(api_key=api_key))

In [20]:
prompt = """You are a animal expert. You need to answer the question related to the type of foxs. 
Given below is the context and question of the user.
context = {context}
question = {question}
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [21]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key = api_key)
retriever = vectorstore.as_retriever()

In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
rag_chain.invoke("What is about the fox red?")

'The red fox is the most common species of fox found throughout the Northern Hemisphere. It is known for its adaptability, intelligence, and ability to thrive in various habitats. The red fox is a skilled hunter, capable of catching a wide range of prey such as rabbits, rodents, and birds. It is a social animal that lives in family groups known as packs. The red fox is a carnivore and part of the Canidae family, with distinctive red fur and a bushy tail used for communication and balance. It is most active at night but can also be seen during the day.'